In [162]:
import os
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from config import password

In [163]:
# Get api data
url = "https://api.covidtracking.com/v1/states/current.json"
response = requests.get(url).json()

In [164]:
#get data for  dataframe
#find out the length of the response so we can use that for the number of states/territories
len(response)
num_state = range(0,56)
#create empty lists
state = []
date = []
death = []
#honestly, was super proud of this part bc i am BAD AT FOR LOOPS but Google/Zed Shaw is my religion now
#create for loop to get the data that we need  for state, date, and death
for i in num_state:
    (print(response[i]["state"]))
    state.append(response[i]["state"])
    (print(response[i]["date"]))
    date.append(response[i]["date"])
    (print(response[i]["deathConfirmed"]))
    death.append(response[i]["death"])

AK
20201025
68
AL
20201025
2680
AR
20201025
1655
AS
20201025
None
AZ
20201025
5586
CA
20201025
None
CO
20201025
None
CT
20201025
3674
DC
20201025
None
DE
20201025
600
FL
20201025
16632
GA
20201025
None
GU
20201025
None
HI
20201025
None
IA
20201025
None
ID
20201025
527
IL
20201025
9505
IN
20201025
3894
KS
20201025
None
KY
20201025
1388
LA
20201025
5631
MA
20201025
9640
MD
20201025
3950
ME
20201025
145
MI
20201025
7182
MN
20201025
2335
MO
20201025
None
MP
20201025
2
MS
20201025
2944
MT
20201025
None
NC
20201025
4097
ND
20201025
340
NE
20201025
None
NH
20201025
None
NJ
20201025
14496
NM
20201025
None
NV
20201025
None
NY
20201025
None
OH
20201025
4896
OK
20201025
None
OR
20201025
None
PA
20201025
None
PR
20201025
610
RI
20201025
None
SC
20201025
3567
SD
20201025
None
TN
20201025
2971
TX
20201025
None
UT
20201025
None
VA
20201025
3329
VI
20201025
21
VT
20201025
58
WA
20201025
2296
WI
20201025
1778
WV
20201025
419
WY
20201025
None


In [165]:
covid_df = pd.DataFrame({"state":state, "date": date, "death":death})
covid_df.head()

,state,date,death
0,AK,20201025,68
1,AL,20201025,2866
2,AR,20201025,1812
3,AS,20201025,0
4,AZ,20201025,5874


In [166]:
# Filepaths
covid_data = os.path.join("data","cdc_covid.csv")
# Read csv into DataFrame
data = pd.read_csv(covid_data)
data.head()
data.shape

(61, 13)

In [167]:
new_data = data.drop(columns = ["Confirmed Cases", "Total Deaths", "Probable Cases", "Cases in Last 7 Days", "Case Rate per 100000", "Confirmed Deaths", "Probable Deaths", "Deaths in Last 7 Days", "Death Rate per 100000", "Case Rate per 100000 in Last 7 Days", "Death Rate per 100K in Last 7 Days"])
new_data.head()
#pd.concat([new_data, covid_data], axis=1)

,State/Territory,Total Cases
0,Alaska,12118
1,Alabama,180916
2,Arkansas,104135
3,American Samoa,0
4,Arizona,235882


In [168]:
new_data.reset_index(inplace=True)
covid_df.reset_index(inplace=True)

In [169]:
new_data.head()

,index,State/Territory,Total Cases
0,0,Alaska,12118
1,1,Alabama,180916
2,2,Arkansas,104135
3,3,American Samoa,0
4,4,Arizona,235882


In [170]:
final_data = pd.merge(new_data, covid_df, on="index", how="outer")

In [171]:
final_data.head()

,index,State/Territory,Total Cases,state,date,death
0,0,Alaska,12118,AK,20201025.0,68.0
1,1,Alabama,180916,AL,20201025.0,2866.0
2,2,Arkansas,104135,AR,20201025.0,1812.0
3,3,American Samoa,0,AS,20201025.0,0.0
4,4,Arizona,235882,AZ,20201025.0,5874.0


In [174]:
final_data.drop(columns=["index"])

,State/Territory,Total Cases,state,date,death
0,Alaska,12118,AK,20201025.0,68.0
1,Alabama,180916,AL,20201025.0,2866.0
2,Arkansas,104135,AR,20201025.0,1812.0
3,American Samoa,0,AS,20201025.0,0.0
4,Arizona,235882,AZ,20201025.0,5874.0
...,...,...,...,...,...
56,Washington,101345,NaN,NaN,NaN
57,Wisconsin,200466,NaN,NaN,NaN
58,West Virginia,21392,NaN,NaN,NaN
59,Wyoming,10545,NaN,NaN,NaN


In [175]:
# Filepaths
state_convert = os.path.join("data","csvData.csv")
# Read csv into DataFrame
state_table = pd.read_csv(state_convert)

In [176]:
state_table.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [177]:
covid_df_state = pd.merge(covid_df, state_table, left_on="state", right_on="Code", how="inner")

In [178]:
covid_df_state.head()

,index,state,date,death,State,Abbrev,Code
0,0,AK,20201025,68,Alaska,Alaska,AK
1,1,AL,20201025,2866,Alabama,Ala.,AL
2,2,AR,20201025,1812,Arkansas,Ark.,AR
3,4,AZ,20201025,5874,Arizona,Ariz.,AZ
4,5,CA,20201025,17345,California,Calif.,CA


In [179]:
final_data = pd.merge(covid_df_state, new_data, left_on="State", right_on="State/Territory", how="inner")

In [180]:
final_data.head()

,index_x,state,date,death,State,Abbrev,Code,index_y,State/Territory,Total Cases
0,0,AK,20201025,68,Alaska,Alaska,AK,0,Alaska,12118
1,1,AL,20201025,2866,Alabama,Ala.,AL,1,Alabama,180916
2,2,AR,20201025,1812,Arkansas,Ark.,AR,2,Arkansas,104135
3,4,AZ,20201025,5874,Arizona,Ariz.,AZ,4,Arizona,235882
4,5,CA,20201025,17345,California,Calif.,CA,5,California,886865


In [181]:
final_data = final_data[["State/Territory", "Code", "death", "Total Cases"]]

In [182]:
final_data.head()
final_data = final_data.rename(columns = {"State/Territory": "state", "Total Cases": "total_cases", "Code": "code"})

In [183]:
# Connect to database
pg_user = "postgres"
pg_password = password
db_name = "covid_ETL"
connection_string = f"{pg_user}:{password}@localhost:5432/{db_name}"
engine = create_engine(f"postgresql://{connection_string}")

In [184]:
engine.table_names()

['covid_table']

In [185]:
final_data.to_sql(name="covid_table",con=engine, if_exists="append", index=False)